In [1]:
#imports
import psycopg2
#import sqlalchemy
#from sqlalchemy import create_engine
#import matplotlib as plt
import pandas as pd
#import geopandas as gpd
import time
%matplotlib inline

In [2]:
#import credentials from cred.py file
from creds import creds

In [10]:
#establish database connection
conn = psycopg2.connect( host=creds['hostname'], user=creds['username'], password=creds['password'], dbname=creds['database'] )
#conn.close()

### Intersection Query

Find the intersection between geolife_data and osm_pois_buffer_ref

In [8]:
script_start = time.time()

test_query = """SELECT *
FROM osm_pois_buffer_ref
limit 1000"""
start = time.time()
intersection_query = """SELECT osm.place_name, osm.fclass, osm.geom as pois_geom, chn.geom AS province_geom, 
(st_intersection (osm.buffer, st_transform(chn.geom,4326))) AS intersection
FROM osm_pois_buffer_ref osm, chn_province_bound chn
limit 10000"""
intersection_frame = pd.read_sql_query(intersection_query ,conn)
elapsed_time_fl_iq = (time.time() - start) 
print('Time for Intersection: {} s'.format(elapsed_time_fl_iq))

Time for Intersection: 207.80536270141602 s


### Spatial Join

Join the osm point buffer geometry to the geolife_data points when they are contained within the buffer

In [12]:
start = time.time()
join_query = """SELECT
  osm.fclass, osm.place_name,osm.buffer,
  geolife.user_id, geolife.active_date, geolife.lat, geolife.lon, geolife.geom
FROM geolife_data AS geolife
JOIN osm_pois_buffer_ref AS osm
ON ST_Contains(osm.buffer, geolife.geom);"""
join_frame = pd.read_sql_query(join_query ,conn)
elapsed_time_fl_sj = (time.time() - start) 
print('Time for Spatial Join: {} s'.format(elapsed_time_fl_sj))

MemoryError: 

### Spatial Clustering
Identify clusters in geolife_data

In [22]:
start = time.time()
#Use ST_ClusterDBSCAN with eps projection unit
#geolife_data projection = 4326, so projection unit in decimal degrees
#This will probably take forever because it's insane
spatial_query = """SELECT *, ST_ClusterDBSCAN(geom, eps := .0005, minPoints := 1000)
                OVER(ORDER BY user_id) AS cluster_id FROM geolife_data
                where user_id='22'
                LIMIT 8;""" 
spatial_clusters = pd.read_sql_query(spatial_query ,conn)
elapsed_time_fl_sc = (time.time() - start) 
print('Time for Spatial Clustering: {} s'.format(elapsed_time_fl_sc))

script_end = time.time()


InterfaceError: connection already closed

In [11]:
#set up a geopandas dataframe from sql query
#osm_pois_gpd = gpd.read_postgis(sql = osm_query, con = conn)
#osm_pois_gpd.head()

In [ ]:
conn.close()